In [45]:
import warnings
warnings.filterwarnings('ignore')

In [46]:
import sys
!{sys.executable} -m pip install lightgbm

In [47]:
import sys
!{sys.executable} -m pip install umap-learn[plot]

In [48]:
import sys
!{sys.executable} -m pip install scikit-learn-intelex

In [49]:
from sklearnex import patch_sklearn
patch_sklearn(global_patch=True)

Scikit-learn was successfully globally patched by Intel(R) Extension for Scikit-learn


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [50]:
import pandas as pd
import numpy as np
#import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [51]:
# Load data and preprocess
df = pd.read_csv('https://gitlab.com/mirsakhawathossain/exodata/-/raw/main/dataset/exodata.csv').sort_index(axis=0)

In [52]:
#df_drop = pd.read_csv('https://gitlab.com/mirsakhawathossain/exodata/-/raw/main/dataset/unrelevant_features.csv').sort_index(axis=0)

In [53]:
#drop_columns = df_drop['feature'].to_list()

In [54]:
columns_1 = df.columns[df.mean()==1]
columns_2 = df.columns[df.mean()==0]
df = df.drop(columns=columns_1)
df = df.drop(columns=columns_2)

In [55]:
#df = df.drop(columns=drop_columns)

In [56]:
df

,PDCSAP_FLUX__has_duplicate_max,PDCSAP_FLUX__has_duplicate_min,PDCSAP_FLUX__has_duplicate,PDCSAP_FLUX__sum_values,PDCSAP_FLUX__abs_energy,PDCSAP_FLUX__mean_abs_change,PDCSAP_FLUX__mean_change,PDCSAP_FLUX__mean_second_derivative_central,PDCSAP_FLUX__median,PDCSAP_FLUX__mean,...,PDCSAP_FLUX__fourier_entropy__bins_5,PDCSAP_FLUX__fourier_entropy__bins_10,PDCSAP_FLUX__fourier_entropy__bins_100,PDCSAP_FLUX__permutation_entropy__dimension_3__tau_1,PDCSAP_FLUX__permutation_entropy__dimension_4__tau_1,PDCSAP_FLUX__permutation_entropy__dimension_5__tau_1,PDCSAP_FLUX__permutation_entropy__dimension_6__tau_1,PDCSAP_FLUX__permutation_entropy__dimension_7__tau_1,PDCSAP_FLUX__mean_n_absolute_max__number_of_maxima_7,exoplanet
0,0.0,0.0,0.0,3.938986e+07,9.572128e+11,153.255300,0.800615,0.149376,24147.1310,24254.842308,...,0.200814,0.485093,1.680648,1.202957,1.721831,2.243674,2.712962,3.118483,25867.303714,0
1,0.0,0.0,1.0,1.010017e+08,2.512213e+12,148.745688,-1.089670,0.186553,24878.2540,24816.135168,...,0.090729,0.300740,1.459540,1.229355,1.768802,2.320843,2.843847,3.322916,26293.361286,0
2,0.0,0.0,1.0,9.922578e+07,2.385546e+12,131.771522,0.197467,0.007730,24192.3595,24002.365215,...,0.515674,0.884632,1.845882,1.372956,2.074437,2.790229,3.472608,4.076345,24832.829429,0
3,0.0,0.0,1.0,9.451435e+07,2.178293e+12,133.917359,-0.217077,-0.045048,23166.6560,23007.389078,...,0.369607,0.779689,1.813868,1.709158,2.899154,4.034073,5.119587,5.996514,24121.541571,0
4,0.0,0.0,1.0,1.073113e+08,2.570948e+12,141.468006,0.066043,-0.023373,24086.9815,23921.372684,...,0.527450,0.878414,1.836681,1.638988,2.603741,3.570002,4.483830,5.222974,24901.026857,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75453,0.0,0.0,1.0,4.711356e+07,5.233879e+11,4.916917,-0.000924,0.000256,11109.1560,11109.068633,...,0.136002,0.245901,2.090807,1.791645,3.176436,4.773906,6.488361,7.851226,11135.958143,1
75454,0.0,0.0,1.0,4.569457e+07,4.892209e+11,4.580461,-0.001654,-0.001213,10706.1080,10706.320227,...,1.132079,1.738595,3.661195,1.789769,3.171699,4.768713,6.479898,7.852670,10753.584286,1
75455,0.0,0.0,1.0,4.555065e+07,4.752317e+11,4.801451,-0.001487,0.001492,10432.9675,10433.038293,...,1.105535,1.769405,3.770656,1.791324,3.175705,4.773828,6.494530,7.878245,10452.424143,1
75456,0.0,0.0,1.0,3.821517e+07,4.132426e+11,4.856287,0.000755,-0.001577,10813.3170,10813.572653,...,1.091023,1.734660,3.707686,1.791595,3.176271,4.773713,6.477202,7.756732,10875.174000,1


In [57]:
X = df.drop(['exoplanet'],axis=1)
y = df[['exoplanet']]

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=43,stratify=y)

In [59]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

In [60]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [61]:
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

In [62]:
#import umap.umap_ as umap
# Define and fit the supervised UMAP model
import umap
supervised_umap = umap.UMAP(n_neighbors=15, n_components=2)
X_train = supervised_umap.fit_transform(X_train,y_train)

In [63]:
# Transform the test data using the trained model
X_test = supervised_umap.transform(X_test)


In [64]:
# Create LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)


In [65]:
# Set hyperparameters for LightGBM model
params = {
    'objective': 'binary',  # specify the objective function for binary classification
    'metric': 'binary_logloss',  # evaluation metric for the model
    'boosting_type': 'gbdt',  # specify the boosting type
    'num_leaves': 31,  # number of leaves in each tree
    'learning_rate': 0.05,  # learning rate of the model
    'feature_fraction': 0.9,  # fraction of features to consider for each tree
    'bagging_fraction': 0.8,  # fraction of data to use for bagging
    'bagging_freq': 5,  # frequency for bagging
    'verbose': 0,  # verbosity level
    'random_state': 42  # random seed for reproducibility
}


In [66]:
# Train the LightGBM model
model = lgb.train(params, train_data, num_boost_round=100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000820 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


In [67]:
# Make predictions on the test set
y_pred = model.predict(X_test)

In [68]:
from sklearn.metrics import accuracy_score

# Convert y_pred to binary
y_pred_binary = [1 if x >= 0.5 else 0 for x in y_pred]

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred_binary)


In [69]:
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 67.71%
